In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import math
from sklearn.neighbors import LocalOutlierFactor

In [2]:
population = pd.read_csv(r"C:\Users\sathv\OneDrive\Desktop\RDM\adult.data.csv")

In [3]:
sample_1 = population

In [4]:
X= sample_1[["age","workclass","education-num","marital-status","occupation","relationship","race","sex","hoursperweek","native-country"]]
y = sample_1["class"]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 42)

In [6]:
X_train

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,hoursperweek,native-country
5514,33,Local-gov,13,Never-married,Prof-specialty,Not-in-family,White,Female,50,United-States
19777,36,Private,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,50,United-States
10781,58,Self-emp-not-inc,5,Separated,Craft-repair,Not-in-family,White,Male,40,United-States
32240,21,Private,11,Married-civ-spouse,Farming-fishing,Husband,White,Male,46,United-States
9876,27,Private,10,Divorced,Other-service,Not-in-family,White,Male,40,United-States
...,...,...,...,...,...,...,...,...,...,...
29802,47,Private,13,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States
5390,31,Private,8,Divorced,Other-service,Unmarried,White,Female,21,United-States
860,18,Private,7,Never-married,Adm-clerical,Own-child,White,Female,20,United-States
15795,50,Self-emp-not-inc,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,84,United-States


In [7]:
y_train

5514       >50K
19777      >50K
10781     <=50K
32240     <=50K
9876      <=50K
          ...  
29802     <=50K
5390      <=50K
860       <=50K
15795     <=50K
23654     <=50K
Name: class, Length: 26048, dtype: object

In [8]:
def detect_outlier(data, col):
    mean = np.mean(data[col])
    std = np.std(data[col])
    threshold = 3
    outlier = []
    index = []
    for i in data.index:
#         print(i)
        z = (data[col][i]-mean)/std
        if z > threshold:
            index.append(i)
            ou = data.at[i, col]
            outlier.append(ou)
#     print(outlier)
    return (list(index),outlier)

In [9]:
def impute(data,col,index):
    new_data = data.copy()
    mode = new_data[col].mode()
    for i in index:
        new_data.at[i, col]=mode
    return new_data
        

In [10]:
def drop_rows(x,only_na):
    new_data = x.copy()
    for i in only_na:
        new_data.drop(i,inplace=True)

    return new_data

In [11]:
index,outlier = detect_outlier(X_train,"age")

In [12]:
X_train = impute(X_train,"age",index)

In [13]:
index,outlier = detect_outlier(X_train,"hoursperweek")

In [14]:
X_train = impute(X_train,"hoursperweek",index)

In [15]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [16]:
def label_encoding(data,cat_cols):
    if cat_cols is None:
        data= label_encoder.fit_transform(data)
        return data
        
    else:
        for i in range(len(cat_cols)):
            data[cat_cols[i]]= label_encoder.fit_transform(data[cat_cols[i]])
        

In [17]:
cols = X_train.select_dtypes([np.object]).columns

C:\Users\sathv\AppData\Local\Temp\ipykernel_29196\2170981920.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cols = X_train.select_dtypes([np.object]).columns


In [18]:
cols

Index(['workclass', 'marital-status', 'occupation', 'relationship', 'race',
       'sex', 'native-country'],
      dtype='object')

In [19]:
label_encoding(X_train,cols)

In [20]:
X_train

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,hoursperweek,native-country
5514,33,2,13,4,10,1,4,0,50,39
19777,36,4,11,2,4,0,4,1,50,39
10781,58,6,5,5,3,1,4,1,40,39
32240,21,4,11,2,5,0,4,1,46,39
9876,27,4,10,0,8,1,4,1,40,39
...,...,...,...,...,...,...,...,...,...,...
29802,47,4,13,2,3,0,4,1,40,39
5390,31,4,8,0,8,4,4,0,21,39
860,18,4,7,4,1,3,4,0,20,39
15795,50,6,9,2,5,0,4,1,40,39


In [21]:
# col = None
y_train = label_encoding(y_train, None)

In [22]:
y_train

array([1, 1, 0, ..., 0, 0, 0])

In [23]:
label_encoding(X_test, cols)

In [24]:
y_test= label_encoding(y_test, None)

In [25]:
y_test

array([0, 0, 1, ..., 1, 0, 0])

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
logisticRegr = LogisticRegression()

In [27]:
def logistic_regression_train(logisticRegr,x,y):
    logisticRegr.fit(x,y)
def logistic_regression_test(logisticRegr,x):
    y_pred = logisticRegr.predict_proba(x)
    return y_pred
def logistic_regression_pred_test(logisticRegr,x):
    y_pred_test = logisticRegr.predict(x)
    return y_pred_test

def average_diff(y_pred, X_test):
    column_i = y_pred.T[0]
    X_test_copy = X_test.copy()
    X_test_copy['preds'] = column_i
    Female = X_test_copy["preds"][X_test_copy["sex"] == 0]
    female_average = sum(Female)/len(Female)
    male = X_test_copy["preds"][X_test_copy["sex"] == 1]
    male_average = sum(male)/len(male)
    diff = (female_average - male_average)
    return diff 

def accuracy(y_test, y_pred_final):
#     y_pred_final = np.argmax(y_pred, axis=1)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_final).ravel()
    true_positive  = tn
    false_positive = fn
    true_negative  = tp
    false_negative = fp
    accuracy = (true_positive + true_negative) / (true_negative + true_positive + false_positive + false_negative )
    return(accuracy)
    
    

In [28]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

y_pred = classifier.predict_proba(X_test)

dirty_diff = average_diff(y_pred,X_test)
dirty_diff

y_pred_test = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix,accuracy_score
dirty_test_score = accuracy_score(y_test,y_pred_test)


C:\Users\sathv\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


# Clean test data

In [29]:
X_clean_train, X_clean_test, y_clean_train, y_clean_test = train_test_split(X, y, test_size=0.20, random_state = 42)

In [30]:
index,outlier= detect_outlier(X_clean_train,"age")

In [31]:
X_clean_train = impute(X_clean_train,"age",index)

In [32]:
index,outlier = detect_outlier(X_clean_train,"hoursperweek")

In [33]:
X_clean_train = impute(X_clean_train,"hoursperweek",index)

In [34]:
index,outlier= detect_outlier(X_clean_test,"age")

In [35]:
X_clean_test = impute(X_clean_test,"age",index)

In [36]:
index,outlier = detect_outlier(X_clean_test,"hoursperweek")

In [37]:
X_clean_test = impute(X_clean_test,"hoursperweek",index)

In [38]:
label_encoding(X_clean_train,cols)

In [39]:
y_clean_train = label_encoding(y_clean_train, None)

In [40]:
label_encoding(X_clean_test,cols)

In [41]:
y_clean_test= label_encoding(y_clean_test, None)

In [42]:
clean_model = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
clean_model.fit(X_clean_train, y_clean_train)

y_clean_pred = classifier.predict_proba(X_clean_test)

clean_diff = average_diff(y_clean_pred,X_clean_test)
clean_diff

y_pred_clean_test = classifier.predict(X_test)

clean_test_score = accuracy_score(y_test,y_pred_test)

C:\Users\sathv\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [43]:
from tabulate import tabulate
data = [["Z-Score","Impute by mode","Dirty test data","Label Encoding", "Logistic Regression", dirty_test_score, dirty_diff],
["Z-Score","Impute by mode", "Clean test data", "Label Encoding", "Logistic Regression", clean_test_score, clean_diff]]
print (tabulate(data, headers=["Detection Method","Method", "Test Data", "Encoding Technique", "ML Model", "Accuracy", "Fairness"]))

Detection Method    Method          Test Data        Encoding Technique    ML Model               Accuracy    Fairness
------------------  --------------  ---------------  --------------------  -------------------  ----------  ----------
Z-Score             Impute by mode  Dirty test data  Label Encoding        Logistic Regression    0.803316    0.177524
Z-Score             Impute by mode  Clean test data  Label Encoding        Logistic Regression    0.803316    0.177111
